## 유튜브의 자막을 추출하고 그에 따라 내용을 요약

In [19]:
!pip install youtube-transcript-api


[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [20]:
!pip install scikit-learn


[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [6]:
from youtube_transcript_api import YouTubeTranscriptApi
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer

video_id='Z-ekdCF5Dvg'
captions = YouTubeTranscriptApi.get_transcript(video_id, languages=['ko', 'en'])
texts = [caption['text'] for caption in captions]

# TF-IDF를 사용하여 각 자막의 중요도 계산
vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(texts)
importance_scores = np.mean(tfidf_matrix, axis=1).A1

# 중요 지점을 감지하기 위한 임계값 설정 (예: 상위 20% 지점을 중요 지점으로 설정)
threshold = np.percentile(importance_scores, 80)

# 중요 지점 추출
important_points = []
for idx, score in enumerate(importance_scores):
    if score >= threshold:
        important_points.append((captions[idx]['start'], captions[idx]['start'] + captions[idx]['duration']))

In [7]:
# 중요 지점의 자막을 추출하고 요약
from dotenv import load_dotenv
from openai import OpenAI

load_dotenv()
client = OpenAI()
summaries = []

for start, end in important_points:
    # 해당 시간 동안의 자막 추출
    segment_texts = [caption['text'] for caption in captions if start <= caption['start'] < end]
    segment_text = ' '.join(segment_texts)
    # GPT-4를 이용해 요약
    response = client.chat.completions.create(
        model='gpt-3.5-turbo',
        temperature=0.5,
        messages=[
            {
                "role": "system",
                "content": "다음과 같이 특정 구간의 자막: {segment_text}을 제시하면 당신은 한글로 내용을 요약해 주고 필요하다면 설명도 추가해줘.",
            },
            {
                "role": "user",
                "content": f"자막: {segment_text}",
            },
        ],
    )

    summary = response.choices[0].message
    summaries.append({'start': start, 'end': end, 'summary': summary})

# 결과 출력
for summary in summaries:
    print(f"중요 지점: {summary['start']}초 ~ {summary['end']}초")
    print(f"요약: {summary['summary'].content}\n")

중요 지점: 3.759초 ~ 10.32초
요약: 이 자막은 "안녕하세요 오늘은 lm2 벡터 라지 랭귀지 모델"이라고 적혀 있어요. 여기서 'lm2 벡터 라지 랭귀지 모델'은 큰 언어 모델을 의미하는 것 같습니다. 이 모델은 자연어 처리 분야에서 사용되며, 텍스트 데이터를 분석하고 이해하는 데 도움을 줍니다.

중요 지점: 26.279초 ~ 31.759초
요약: 이 자막은 "같습니다 먼저 어 디코더 언니 모델을 이용한 텍 베딩에 대해서 잠깐의"라는 문장을 나타냅니다.

이 문장은 자연어 처리 분야에서 사용되는 딥러닝 모델인 어텐션(attention) 메커니즘을 활용한 디코더(Decoder) 모델과 텍스트 임베딩(Text Embedding)에 대한 간략한 언급을 하고 있는 것으로 보입니다. 이는 자연어 처리 모델에서 텍스트를 벡터 형태로 변환하여 처리하는 과정에 관한 내용일 수 있습니다.

중요 지점: 34.719초 ~ 40.239000000000004초
요약: 이 자막은 자연어 처리 분야에서 사용되는 텍스트 임베딩에 대한 이야기를 하고 있습니다. "lm2 벡터"에 대한 논문을 다루는 것으로 보이는데, 이 논문은 최신 연구나 모델에 대한 내용을 다루고 있을 것입니다. 텍스트 임베딩은 단어나 문장을 벡터 형태로 표현하는 기술로, 자연어 처리 모델의 성능을 향상시키는 데 중요한 역할을 합니다. "lm2 벡터"에 대한 논문을 살펴보면 해당 벡터의 특징이나 활용 방법 등에 대한 정보를 얻을 수 있을 것입니다.

중요 지점: 37.559초 ~ 43.28초
요약: lm2 벡터 논문에 대한 세미나 주제를 살펴본 후, 해당 논문의 내용을 이해하고 있습니다.

중요 지점: 59.239초 ~ 64.39999999999999초
요약: 이 자막은 "버트 모델"에 대한 언급이 있습니다. 버트 모델은 자연어 처리를 위한 트랜스포머 모델 중 하나로, 최근에 많은 주목을 받고 있는 모델입니다. 이번 내용은 버트 모델에 대한 도움이 될 수 있다는 내용으로 해석될 수 있습니다. 이 자막은 버트 